In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.tree import _tree

Прочитаем два датасета об употреблении школьниками алкоголя и объединим их:

In [2]:
df1 = pd.read_csv('student-mat.csv')
df2 = pd.read_csv('student-por.csv')
df = pd.concat([df1, df2])
df.reset_index(drop=True, inplace=True)

In [3]:
df.values.shape

(1044, 33)

Как видно, в датасете 33 признака и выборка имеет размер 1044. Посмотрим на признаки:

In [4]:
for i in df.columns:
    arr = df[i].unique()
    arr.sort()
    print(i, arr)

school ['GP' 'MS']
sex ['F' 'M']
age [15 16 17 18 19 20 21 22]
address ['R' 'U']
famsize ['GT3' 'LE3']
Pstatus ['A' 'T']
Medu [0 1 2 3 4]
Fedu [0 1 2 3 4]
Mjob ['at_home' 'health' 'other' 'services' 'teacher']
Fjob ['at_home' 'health' 'other' 'services' 'teacher']
reason ['course' 'home' 'other' 'reputation']
guardian ['father' 'mother' 'other']
traveltime [1 2 3 4]
studytime [1 2 3 4]
failures [0 1 2 3]
schoolsup ['no' 'yes']
famsup ['no' 'yes']
paid ['no' 'yes']
activities ['no' 'yes']
nursery ['no' 'yes']
higher ['no' 'yes']
internet ['no' 'yes']
romantic ['no' 'yes']
famrel [1 2 3 4 5]
freetime [1 2 3 4 5]
goout [1 2 3 4 5]
Dalc [1 2 3 4 5]
Walc [1 2 3 4 5]
health [1 2 3 4 5]
absences [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 30 32 38 40 54 56 75]
G1 [ 0  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
G2 [ 0  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
G3 [ 0  1  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


Следующая функция печатает решающее дерево:

In [5]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

Построим матрицу признаков в вещественных числах:

In [6]:
X = df.copy()
X.drop(['Dalc', 'Walc', 'Mjob', 'Fjob', 'reason', 'guardian'], axis=1, inplace=True)
names = X.columns.values

X.replace('GT3', 0, inplace=True)
X.replace('LE3', 1, inplace=True)

X.replace('GP', 0, inplace=True)
X.replace('MS', 1, inplace=True)

X.replace('no', 0, inplace=True)
X.replace('yes', 1, inplace=True)

X.replace('F', 0, inplace=True)
X.replace('M', 1, inplace=True)

X.replace('R', 0, inplace=True)
X.replace('U', 1, inplace=True)

X.replace('A', 0, inplace=True)
X.replace('T', 1, inplace=True)

X = X.values.astype(np.double)

Попробуем бинаризовать признаки Walc и Dalc (употребление алкоголя в выходные и в будни) с разным порогом и посмотрим, насколько хорошо решающее дерево может различить два класса людей. Посмотрим также на F1-меру.

In [7]:
def count_score(X, feature, threshold):
    y = (df[feature] > threshold).values.astype(np.int)
    iters = 100
    score = np.array([0.0, 0.0])
    for _ in range(iters):
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
        model = tree.DecisionTreeClassifier()
        model.fit(X_train, y_train)

        for class_number in [0, 1]:
            X_test_one_class, y_test_one_class = X_test[y_test == class_number], y_test[y_test == class_number]
            score[class_number] += (model.predict(X_test_one_class) == y_test_one_class).sum() / y_test_one_class.size
    return score / iters


for feature in ['Walc', 'Dalc']:
    print(feature)
    for threshold in range(1, 5):
        score = count_score(X, feature, threshold)
        print(threshold, score.round(decimals=2), (2 / (1 / score[0] + 1 / score[1])).round(decimals=2))

Walc
1 [0.61 0.75] 0.68
2 [0.75 0.65] 0.7
3 [0.88 0.59] 0.71
4 [0.95 0.48] 0.63
Dalc
1 [0.82 0.6 ] 0.69
2 [0.91 0.41] 0.56
3 [0.96 0.29] 0.44
4 [0.98 0.29] 0.45


Вышло достаточно неплохо! Для выходных получается предсказывать употребление алкоголя чуть лучше, чем для будних дней. Посмотрим на неглубокое решающее дерево для порога 3 и выходных дней:

In [8]:
y = (df['Walc'] > 3).values.astype(np.int)
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(X, y)
tree_to_code(model, names)

def tree(school, sex, age, address, famsize, Pstatus, Medu, Fedu, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, health, absences, G1, G2, G3):
  if goout <= 3.5:
    if sex <= 0.5:
      if studytime <= 1.5:
        return [[56. 10.]]
      else:  # if studytime > 1.5
        return [[307.   7.]]
    else:  # if sex > 0.5
      if Medu <= 0.5:
        return [[0. 2.]]
      else:  # if Medu > 0.5
        return [[233.  39.]]
  else:  # if goout > 3.5
    if sex <= 0.5:
      if famrel <= 3.5:
        return [[29. 17.]]
      else:  # if famrel > 3.5
        return [[144.  21.]]
    else:  # if sex > 0.5
      if famrel <= 3.5:
        return [[ 2. 34.]]
      else:  # if famrel > 3.5
        return [[62. 81.]]


Самый важный признак в модели --- goout, затем sex. Ещё сильно влияет studytime и famrel. Посмотрим, что происходит с будними днями (порог будет 2):

In [9]:
y = (df['Dalc'] > 2).values.astype(np.int)
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(X, y)
tree_to_code(model, names)

def tree(school, sex, age, address, famsize, Pstatus, Medu, Fedu, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, health, absences, G1, G2, G3):
  if sex <= 0.5:
    if age <= 18.5:
      if freetime <= 3.5:
        return [[378.   7.]]
      else:  # if freetime > 3.5
        return [[153.  14.]]
    else:  # if age > 18.5
      if goout <= 4.5:
        return [[30.  3.]]
      else:  # if goout > 4.5
        return [[2. 4.]]
  else:  # if sex > 0.5
    if goout <= 3.5:
      if absences <= 12.5:
        return [[239.  25.]]
      else:  # if absences > 12.5
        return [[4. 6.]]
    else:  # if goout > 3.5
      if G1 <= 12.5:
        return [[83. 54.]]
      else:  # if G1 > 12.5
        return [[34.  8.]]


Тут опять сильно влияет пол и goout. Забавно, кстати, что тут ещё есть признаки, связанные с учёбой: прогулы (absences) и результаты какого-то экзамена (G1). На употребление алкоголя в выходные они не так сильно влияли.